In [4]:
# Import necessary libraries
import os
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
from ipywidgets import Button, VBox, HBox, Output
from IPython.display import display

# Define paths
image_folder = r"C:\Users\User\Desktop\PCB Detection\dataset\test\images"
model_path = r"C:\Users\User\Desktop\PCB Detection\runs\train\exp\weights\best.pt"

# Load the YOLO model
model = YOLO(model_path)

# Get all image files in the folder
image_files = [f for f in os.listdir(image_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
image_files.sort()  # Sort images alphabetically
current_index = [0]  # Use a list for mutable index tracking

# Create widgets
btn_prev = Button(description="Previous", button_style='info')
btn_next = Button(description="Next", button_style='info')
output = Output()

# Function to display image with YOLOv8 results inside Jupyter Notebook
def display_image(index):
    # Clear previous output
    output.clear_output(wait=True)
    with output:
        # Get the current image path
        img_path = os.path.join(image_folder, image_files[index])
        
        # Run YOLOv8 model prediction
        results = model.predict(img_path, save=False, imgsz=640)
        
        # Load the image with OpenCV
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB for Jupyter display
        
        # Draw bounding boxes on the image using Matplotlib
        plt.figure(figsize=(10, 10))
        plt.imshow(img)
        
        for box, conf, cls in zip(results[0].boxes.xyxy, results[0].boxes.conf, results[0].boxes.cls):
            x1, y1, x2, y2 = map(int, box)  # Extract bounding box coordinates
            label = f"{model.names[int(cls)]} {conf:.2f}"  # Class label and confidence
            
            # Draw bounding box
            plt.gca().add_patch(plt.Rectangle(
                (x1, y1), x2 - x1, y2 - y1, fill=False, edgecolor='lime', linewidth=2
            ))
            # Draw label
            plt.text(x1, y1 - 10, label, color='lime', fontsize=12, backgroundcolor="black")
        
        plt.axis('off')
        plt.title(f"Prediction for: {image_files[index]}")
        plt.show()

# Button callbacks
def on_prev_clicked(b):
    if current_index[0] > 0:
        current_index[0] -= 1
        display_image(current_index[0])

def on_next_clicked(b):
    if current_index[0] < len(image_files) - 1:
        current_index[0] += 1
        display_image(current_index[0])

# Attach button callbacks
btn_prev.on_click(on_prev_clicked)
btn_next.on_click(on_next_clicked)

# Initial display
display_image(current_index[0])

# Display the buttons and output
display(VBox([HBox([btn_prev, btn_next]), output]))
